# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

In [5]:
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

## Create the vector store

In [6]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

llm = OpenAI(temperature=0)

In [7]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [8]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [9]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [10]:
from langchain_community.document_loaders import WebBaseLoader

In [11]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [13]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

## Create the Agent

In [15]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [16]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [17]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/Users/ricky/study/langchain-cookbook/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)

/Users/ricky/study/langchain-cookbook/.venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden did not mention Ketanji Brown Jackson in the state of the union address.
Thought: I should use the Ruff QA System to check if there are any errors in the question.
Action: Ruff QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should rephrase the question to make sure it is grammatically correct.
Action: Ruff QA System
Action Input: "What was Biden's statement about Ketanji Brown Jackson in the state of the union address?"
Observation:  I don't know.
Thought: I should try using different keywords to see if I can get a better answer.
Action: State of Union QA System
Action Input: "What did Biden mention about Ketanji Brown Jackson in the

'Biden did not mention Ketanji Brown Jackson in the state of the union address.'

In [19]:
agent.run("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it may be useful in certain situations.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff can replace multiple Flake8 plugins, has a larger rule set, and can automatically fix its own lint violations. It also supports linting for any Python version from 3.7 onwards, while Flake8 only supports up to Python 3.9. Additionally, Ruff does not require the installation of Rust, making it easier to use.
Thought: This information is helpful, but I should also consider the specific needs of my project before deciding which linter to use.
Action: State of Union QA System
Action Input: "What are the unique features of ruff compared to flake8?"
Observation:  I don't know.
Thought: I should also research the specific features and capabilities of flake8 to fully understand the differences between the two linters.
Action: Ruff QA System
Action Input: "

'Ruff has unique features such as automatic fixing of lint violations, support for Python 3.12, and no need to install Rust, making it a useful alternative to flake8 in certain situations. However, the specific needs of the project should also be considered before deciding which linter to use.'

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [20]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [21]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [22]:
agent.run(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden did not mention Ketanji Brown Jackson in the state of the union address.


> Finished chain.


'\nBiden did not mention Ketanji Brown Jackson in the state of the union address.'

In [23]:
agent.run("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it's worth exploring the differences.
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation:  Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8's ~409. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has some minor incompatibilities with Flake8 plugins. However, many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, while Flake8 does not. Ruff is also c

' Ruff and Flake8 are both linters for Python code, but they have some key differences. Ruff uses different rule codes and prefixes than Flake8, which helps minimize conflicts across plugins and allows for easier toggling of individual plugins. Ruff also has a larger rule set, with over 800 rules compared to Flake8\'s ~409. However, Ruff does not support custom lint rules, while Flake8 does. Additionally, Ruff is not a "pure" drop-in replacement for Flake8, as they enforce different sets of rules. Ruff also has some minor incompatibilities with Flake8 plugins. However, many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, while Flake8 does not. Ruff is also compatible with Black, isort, yesqa, eradicate, and most of the rules implemented in pyupgrade, while Flake8 is not. Ruff can also be used as a formatter, while Flake8 cannot. Ruff supports Python 3.7 onwards, while Flake8 supports Python 2. Ruff is also av

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [24]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [25]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [26]:
agent.run(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should check if the president mentioned the tool in the state of the union before answering the first question.
Action: State of Union QA System
Action Input: What tool did the president mention in the state of the union?
Observation:  Economic sanctions, a dedicated task force, and joining with European allies to find and seize assets of Russian oligarchs.
Thought: This is not the tool mentioned in the question.
Action: State of Union QA System
Action Input: Did the president mention any tools in the state of the union?
Observation:  Yes, the president mentioned using economic sanctions and cutting off access to technology as tools to inflict pain on Russia and support Ukraine.
Thought: This could be the tool mentioned in the question.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation:  Ruff integrates with nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Though

'The tool that ruff uses to run over Jupyter Notebooks is nbQA, but the president did not mention it in the state of the union.'